In [1]:
#Importing Necessary Libraries
import json, math, pandas as pd
from collections import defaultdict

In [2]:

def safe_str(x):
    return str(x) if x is not None else ''

def safe_float(x):
    if x is None:
        return 0.0
    if isinstance(x, dict):
        for v in x.values():
            try:
                return float(v)
            except (ValueError, TypeError):
                continue
        return 0.0
    try:
        return float(x)
    except (ValueError, TypeError):
        return 0.0

def load(path):
    with open(path, encoding='utf-8') as f:
        raw = json.load(f)
    # Accept list or ND-json
    return raw if isinstance(raw, list) else [json.loads(l) for l in f if l.strip()]

In [3]:
# Feature engineering

def build_features(df):
    """Robust feature builder that auto-detects column names & flattens nested dicts."""
    # auto-detect columns 
    addr_cols = [c for c in df.columns if df[c].astype(str).str.startswith('0x').any()]
    if not addr_cols:
        raise KeyError("No column contains Ethereum addresses (0x...)")
    addr_col = addr_cols[0]

    action_col = next((c for c in ['action', 'event', 'type'] if c in df.columns), 'action')
    amount_col = next((c for c in ['amount', 'value', 'amountUSD'] if c in df.columns), 'amount')
    asset_col  = next((c for c in ['asset', 'reserve', 'symbol', 'assetSymbol', 'token'] if c in df.columns), 'asset')

    feats = defaultdict(lambda: defaultdict(float))

    for _, r in df.iterrows():
        # flatten nested dicts once
        flat = dict(r)
        if isinstance(flat.get('actionData'), dict):
            flat.update(flat['actionData'])

        u   = safe_str(flat.get(addr_col))
        a   = safe_str(flat.get(action_col, flat.get('type'))).lower()
        amt = safe_float(flat.get(amount_col, flat.get('amount')))
        ts  = int(flat.get('timestamp', 0) or 0)
        tok = safe_str(flat.get(asset_col, flat.get('assetSymbol', 'unknown')))

        # accumulate per action 
        if a == 'deposit':
            feats[u]['total_deposited'] += amt
            feats[u]['deposit_cnt']     += 1
            feats[u]['first_tx'] = min(feats[u]['first_tx'], ts) if 'first_tx' in feats[u] else ts
            feats[u]['last_tx']  = max(feats[u]['last_tx'], ts)  if 'last_tx'  in feats[u] else ts
        elif a == 'borrow':
            feats[u]['total_borrowed'] += amt
            feats[u]['borrow_cnt']     += 1
        elif a == 'repay':
            feats[u]['total_repaid'] += amt
            feats[u]['repay_cnt']    += 1
        elif a == 'redeemunderlying':
            feats[u]['total_redeemed'] += amt
        elif a == 'liquidationcall':
            feats[u]['liquidated_cnt'] += 1
            feats[u]['liquidated_vol'] += amt

    # final feature dict
    out = {}
    for u, f in feats.items():
        age_days = max((f.get('last_tx', 0) - f.get('first_tx', 0)) / 86400, 1)
        dep = f.get('total_deposited', 1e-9)
        bor = f.get('total_borrowed', 1e-9)
        ratio = dep / (dep + bor + 1e-9)
        repay_ratio = f.get('total_repaid', 0) / (bor + 1e-9)
        liq_penalty = min(f.get('liquidated_cnt', 0) * 50, 200)
        avg_dep = f.get('total_deposited', 0) / max(f.get('deposit_cnt', 1), 1)
        div = len(set(df[df[addr_col] == u][asset_col])) if asset_col in df.columns else 1
        out[u] = {
            'ratio': ratio,
            'repay_ratio': min(repay_ratio, 1),
            'liq_penalty': liq_penalty,
            'avg_dep': math.log1p(avg_dep),
            'diversity': min(div, 10),
            'age_days': age_days
        }
    return out

In [4]:
#  Scoring formula

def score(features):
    """Map engineered features to 0-1000 integer."""
    f = features
    base = 0
    base += int(f['ratio'] * 400)
    base += int(f['repay_ratio'] * 300)
    base += int(min(f['diversity']/5, 1) * 100)
    base += int(min(f['age_days']/365, 1) * 100)
    base += int(min(f['avg_dep']/10, 1) * 100)
    base = max(base - f['liq_penalty'], 0)
    return int(min(max(base, 0), 1000))

In [ ]:
#Main Funtion
def run():
    df = pd.DataFrame(load('user-wallet-transactions.json'))
    feats = build_features(df)
    final_scores = {u: score(f) for u, f in feats.items()}
    with open('scores.json', 'w', encoding='utf-8') as f:
        json.dump(final_scores, f, indent=2)
    print(f"Scored {len(final_scores)} wallets → scores.json")

run()

Scored 3497 wallets → scores.json
